In [1]:
# R to Python conversion of GGM simulation setup
import numpy as np

# Set parameters (matching R code)
n = 2000
p = 200
edge_strength = -0.8
graph_type = "band"

def generate_precision_matrix(p, b, graph_type, random_state=None):
    """Generate precision matrix for different graph structures."""
    if random_state is not None:
        np.random.seed(random_state)
    
    Omega_0 = np.zeros((p, p))
    
    if graph_type == "band":
        np.fill_diagonal(Omega_0, 1)
        for i in range(p - 1):
            for j in range(i + 1, min(p, i + 11)):
                Omega_0[i, j] = np.sign(b) * np.abs(b) ** (abs(i - j) / 10)
                Omega_0[j, i] = Omega_0[i, j]
    elif graph_type == "block":
        for k in range(0, p, 20):
            block_size = min(20, p - k)
            Omega_0[k:k+block_size, k:k+block_size] = b
            np.fill_diagonal(Omega_0[k:k+block_size, k:k+block_size], 1)
    elif graph_type == "erdos":
        np.fill_diagonal(Omega_0, 1)
        for i in range(p - 1):
            for j in range(i + 1, p):
                if np.random.binomial(1, 0.1):
                    phi_ij = np.random.uniform(-0.6, -0.2) if np.random.rand() < 0.5 else np.random.uniform(0.2, 0.6)
                    Omega_0[i, j] = Omega_0[j, i] = phi_ij
    elif graph_type == "cluster":
        for k in range(0, p, 40):
            block_size = min(40, p - k)
            np.fill_diagonal(Omega_0[k:k+block_size, k:k+block_size], 1)
            for i in range(k, k + block_size - 1):
                for j in range(i + 1, k + block_size):
                    if np.random.binomial(1, 0.5):
                        phi_ij = np.random.uniform(-0.6, -0.2) if np.random.rand() < 0.5 else np.random.uniform(0.2, 0.6)
                        Omega_0[i, j] = Omega_0[j, i] = phi_ij
    
    eigenvals = np.linalg.eigvals(Omega_0)
    lambda_min = np.min(eigenvals)
    Omega = Omega_0 + (abs(lambda_min) + 0.5) * np.eye(p)
    return Omega

Omega = generate_precision_matrix(p, edge_strength, graph_type, random_state=42)
true_support = (np.abs(Omega) > 1e-10).astype(int)
upper_tri_mask = np.triu(np.ones_like(true_support), k=1).astype(bool)
true_labels = true_support[upper_tri_mask]
Signal_index = np.where(true_labels == 1)[0]

np.random.seed(42)
Sigma = np.linalg.inv(Omega)
X = np.random.multivariate_normal(np.zeros(p), Sigma, size=n)
X = (X - X.mean(axis=0)) / X.std(axis=0)

print(f"Data shape: X={X.shape}")
print(f"Graph type: {graph_type}")
print(f"Total edges: {p * (p - 1) // 2}")
print(f"True signal edges: {len(Signal_index)}")
print(f"Sparsity: {len(Signal_index) / (p * (p - 1) // 2):.3f}")


Data shape: X=(2000, 200)
Graph type: band
Total edges: 19900
True signal edges: 1945
Sparsity: 0.098


In [2]:
from nullstrap.models.ggm import NullstrapGGM

model = NullstrapGGM(
    fdr=0.1,
    alpha_=None,
    B_reps=5,
    selection_method='cv',
    random_state=42
)

model.fit(X)

print(f"Threshold: {model.threshold_}")
print(f"Selected edges: {model.selected_[:20]}...")
print(f"Number selected: {len(model.selected_)}")
print(f"Correction factor: {model.correction_factor_}")
print(f"Alpha used: {model.alpha_used_}")


Threshold: 0.0702217809582559
Selected edges: [   0    3    8  398  399  400  404  406  597  598  599  600  790  794
  813  990 1179 1181 1184 1188]...
Number selected: 956
Correction factor: 0.050000009863306466
Alpha used: 0.01778240153111564


In [3]:
from sklearn.covariance import GraphicalLassoCV

glasso = GraphicalLassoCV(cv=5).fit(X)
glasso_precision = glasso.precision_
upper_tri_indices = np.triu_indices(p, k=1)
glasso_edges = np.abs(glasso_precision[upper_tri_indices]) > 1e-6
glasso_selected = np.where(glasso_edges)[0]

print(f"GraphicalLasso selected: {glasso_selected[:20]}...")
print(f"Number selected: {len(glasso_selected)}")
print(f"Alpha used: {glasso.alpha_}")


GraphicalLasso selected: [  0   3   5   7   8   9 199 200 201 203 204 205 206 207 208 212 398 399
 400 401]...
Number selected: 2404
Alpha used: 0.0992248470233921


/home/wbguo/apps/anaconda3/envs/dev/lib/python3.12/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [4]:
from nullstrap.utils.metrics import compute_fdr, compute_power

# Nullstrap GGM metrics
nullstrap_fdr = compute_fdr(model.selected_, Signal_index, len(true_labels))
nullstrap_power = compute_power(model.selected_, Signal_index)

print(f"Nullstrap GGM Power: {nullstrap_power:.3f}")
print(f"Nullstrap GGM FDR: {nullstrap_fdr:.3f}")

# GraphicalLasso metrics
glasso_fdr = compute_fdr(glasso_selected, Signal_index, len(true_labels))
glasso_power = compute_power(glasso_selected, Signal_index)

print(f"GraphicalLasso Power: {glasso_power:.3f}")
print(f"GraphicalLasso FDR: {glasso_fdr:.3f}")


Nullstrap GGM Power: 0.485
Nullstrap GGM FDR: 0.014
GraphicalLasso Power: 0.980
GraphicalLasso FDR: 0.207
